In [6]:
from data import OptiverDataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
import os

In [7]:
class OptiverLSTM(nn.Module):
    def __init__(self, hidden_size=128, layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size=14, 
                            hidden_size=hidden_size, 
                            num_layers=layers, 
                            batch_first=True, 
                            dropout=dropout,
                            proj_size=1
                           )
    def forward(self, x):
        x, _ = self.lstm(x)
        return x

In [8]:
def collate_fn(data):
    return torch.stack([item[0] for item in data]), torch.stack([item[1] for item in data])

In [5]:
def train_lstm(hidden_size, layers, dropout, num_epochs=5): 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = OptiverLSTM(hidden_size=hidden_size, layers=layers, dropout=dropout)
    model = model.to(device)
    
    batch_size = 1
    
    train_dataset = OptiverDataset(split='train')
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    val_dataset = OptiverDataset(split='val')
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    
    maes = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}')
        model.train()
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
                
        model.eval()
        loss = 0
        num_items = 0
        progress = tqdm(total=len(val_dataloader), desc='Validating')
        for x, y in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss += torch.sum(torch.absolute(y_pred - y)).item()
            num_items += torch.numel(y_pred)
            progress.update()
        progress.close()
        mae = loss/num_items
        maes.append(mae)
        print(f'MAE = {mae}')
    return min(maes)


In [19]:
import pandas as pd
hidden_sizes = [64, 128, 256]
layers = [2, 4, 6, 8]
dropouts = [0.0, 0.1, 0.2, 0.3, 0.4]

hidden_size_df = pd.DataFrame(columns=['hidden_size', 'mae'])
layer_df = pd.DataFrame(columns=['num_layers', 'mae'])
dropout_df = pd.DataFrame(columns=['dropout', 'mae'])

for hidden_size in hidden_sizes:
    mae = train_lstm(hidden_size=hidden_size, layers=4, dropout=0)
    hidden_size_df = hidden_size_df.append({'hidden_size': hidden_size, 'mae': mae}, ignore_index=True)

for num_layers in layers:
    mae = train_lstm(hidden_size=128, layers=num_layers, dropout=0)
    layer_df = layer_df.append({'num_layers': num_layers, 'mae': mae}, ignore_index=True)

for dropout in dropouts:
    mae = train_lstm(hidden_size=128, layers=4, dropout=dropout)
    dropout_df = dropout_df.append({'dropout': dropout, 'mae': mae}, ignore_index=True)

hidden_size_df
layer_df
dropout_df


Loading data: 100%|██████████| 139/139 [00:00<00:00, 256.79it/s]

Loading data: 100%|██████████| 20/20 [00:00<00:00, 214.17it/s]

Epoch 1


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 6.014926160828354
Epoch 2


...

MAE = 6.011546727761421
Epoch 5


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 5.999076419373895


,dropout,mae
0,0.0,6.014987
1,0.1,6.014920
2,0.2,5.984556
3,0.3,6.014943
4,0.4,5.999076


In [20]:
hidden_size_df

,hidden_size,mae
0,64.0,5.983188
1,128.0,6.014933
2,256.0,5.990259


In [21]:
layer_df

,num_layers,mae
0,2.0,5.977934
1,4.0,6.015016
2,6.0,6.014881
3,8.0,6.014901


In [22]:
dropout_df

,dropout,mae
0,0.0,6.014987
1,0.1,6.014920
2,0.2,5.984556
3,0.3,6.014943
4,0.4,5.999076


In [ ]:
def test():
    model = OptiverLSTM(hidden_size=64, layers=2, dropout=0.25)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    train_dataset = OptiverDataset(split='train_val')
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    model.train()
    for i in range(5):
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
        
    test_dataset = OptiverDataset(split='test')
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    
    model = model.to(device)
    model.eval()
    loss = 0
    num_items = 0
    progress = tqdm(total=len(test_dataloader), desc='Testing')
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x).squeeze(dim=2)
        loss += torch.sum(torch.absolute(y_pred - y)).item()
        num_items += torch.numel(y_pred)
        progress.update()
    progress.close()
    mae = loss/num_items
    return mae
mae = test()

Loading data: 100%|██████████| 159/159 [00:00<00:00, 281.48it/s]


Training:   0%|          | 0/159 [00:00<?, ?it/s]

In [ ]:
mae